In [4]:
from download_websites import download_websites
from generate_scraping_tasks import generate_scraping_tasks
from generate_solution_code import generate_solution_code
from extract_html_info import extract_relevant_information

import pandas as pd
import os
import json

In [5]:
websites_csv = 'websites_evaluation.csv'
save_folder = 'downloaded_pages'
ROWS = (0, 0)
NUM_TASKS_PER_WEBSITE = 10

# Download websites
# download_websites(websites_csv, save_folder)

# Generate scraping tasks and solutions
websites = pd.read_csv(websites_csv)

for i, row in websites.iterrows():
    if i < ROWS[0] or i > ROWS[1]:
        continue
    
    print("--------------------")

    category, website, link = row
    HTML_file = f'{save_folder}/{website}.html'

    if not os.path.exists(HTML_file):
        print(f'{website}.html does not exist')
        continue

    print(f"Generating data for {website}")

    # Check if relevant information has already been extracted
    if os.path.exists(f'extracted_info/{website}.txt'):
        with open(f'extracted_info/{website}.txt', 'r') as f:
            HTML_elements = f.read()
        print(f'Loaded HTML elements for {website}')
    else:
        HTML_elements = extract_relevant_information(HTML_file)
        with open(f'extracted_info/{website}.txt', 'w') as f:
            f.write(HTML_elements)
        print(f'Extracted HTML elements for {website}')
    

    # Check if scraping tasks have already been generated
    if not os.path.exists(f'scraping_tasks/{website}.txt'):
        generate_scraping_tasks(link, website, category, HTML_elements, NUM_TASKS_PER_WEBSITE)
        print(f'Generated scraping tasks for {website}')
    with open(f'scraping_tasks/{website}.txt', 'r') as f:
        scraping_tasks = pd.DataFrame([{'task': task} for task in f.readlines()])
        print(f'Loaded scraping tasks for {website}')

    # Check if the solution code folder has already been generated
    if not os.path.exists(f'solution_code/{website}'):
        os.mkdir(f'solution_code/{website}')
        print(f'Created solution code folder for {website}')

    for j, task_description in scraping_tasks.iterrows():
        task_description = task_description['task']
        if not os.path.exists(f'solution_code/{website}/{website}_{j}.py'):
            generate_solution_code(website, HTML_file, category, HTML_elements, task_description, j)
            print(f'Generated solution for {website}_{j}')
        else:
            print(f'Solution for {website}_{j} already exists')
    
    print(f'Generated all solutions for {website}')

print("--------------------")

--------------------
Generating data for airbnb
Loaded HTML elements for airbnb
Loaded scraping tasks for airbnb
Solution for airbnb_0 already exists
Generated solution for Save the classes of the "show more" button in seperate rows in a CSV file.

Solution for airbnb_2 already exists
Solution for airbnb_3 already exists
Solution for airbnb_4 already exists
Solution for airbnb_5 already exists
Solution for airbnb_6 already exists
Solution for airbnb_7 already exists
Solution for airbnb_8 already exists
Solution for airbnb_9 already exists
Generated all solutions for airbnb
--------------------


In [ ]:
# Combine all scraping tasks and solutions into one file

# Load prompts
system_prompt = open('prompts/generate_solution_code_system_prompt.txt', 'r').read()
user_prompt = open('prompts/generate_solution_code_user_prompt.txt', 'r').read()

# Load extracted info
extracted_info = {}
for filename in os.listdir('extracted_info'):
    with open(f'extracted_info/{filename}', 'r') as f:
        extracted_info[filename.split('.')[0]] = f.read()
    
# Load scraping tasks
scraping_tasks = {}
for filename in os.listdir('scraping_tasks'):
    with open(f'scraping_tasks/{filename}', 'r') as f:
        scraping_tasks[filename.split('.')[0]] = f.readlines()

# Load solutions
solutions = {}
for folder in os.listdir('solution_code'):
    solutions[folder] = {}
    for filename in os.listdir(f'solution_code/{folder}'):
        with open(f'solution_code/{folder}/{filename}', 'r') as f:
            solutions[folder][filename.split('.')[0]] = f.read()

# Load website infos
websites = pd.read_csv('websites_validation.csv')

# Generate prompts
prompts = {} # Dictionary of dictionaries
categories = {}
links = {}
for i, row in websites.iterrows():
    category, website, link = row
    categories[website] = category
    links[website] = link
    if website not in scraping_tasks:
        print(f'{website} does not have generated data')
        continue
    scraping_tasks_website = scraping_tasks[website]
    HTML_file = f'downloaded_pages/{website}.html'
    prompts[website] = {}
    for j, scraping_task in enumerate(scraping_tasks_website):
        scraping_task = scraping_task.strip()
        solution = solutions[website][f'{website}_{j}']
        HTML_elements = extracted_info[website]
        prompts[website][f'{website}_{j}'] = user_prompt.format(website=website, HTML_file=HTML_file, category=category, HTML_string=HTML_elements, task=scraping_task)

# Generate training samples
training_samples = {}
sample_template = """### System:
{system_prompt}

### User:
{user_prompt}

### Response:
```
{response}
```
"""
system_prompt = open('prompts/generate_solution_code_system_prompt.txt', 'r').read()
for website in prompts:
    training_samples[website] = {}
    for task_description in prompts[website]:
        sample = sample_template.format(system_prompt=system_prompt, user_prompt=prompts[website][task_description], response=solutions[website][task_description])
        training_samples[website][task_description] = sample

# Convert prompts, solutions and training samples to lists
websites_list = []
task_names_list = []
categories_list = []
links_list = []
prompts_list = []
solutions_list = []
training_samples_list = []
for website in prompts:
    for task_description in prompts[website]:
        websites_list.append(website)
        task_names_list.append(task_description)
        categories_list.append(categories[website])
        links_list.append(links[website])
        prompts_list.append(prompts[website][task_description])
        solutions_list.append(solutions[website][task_description])
        training_samples_list.append(training_samples[website][task_description])

# Combine to a list of dictionaries
data = []
for i in range(len(prompts_list)):
    data.append({
        'website': websites_list[i],
        'task': task_names_list[i],
        'category': categories_list[i],
        'link': links_list[i],
        'prompt': prompts_list[i],
        'solution': solutions_list[i],
        'training_sample': training_samples_list[i]
    })
print("Dataset length:", len(data))

# Save prompts, solutions and training samples as a json file
# where each 'row' is a prompt, solution and training sample
# for a particular website and task
with open('dataset_validation.json', 'w') as f:
    json.dump(data, f)

In [ ]:
import matplotlib.pyplot as plt

# Load dataset from json file
with open('dataset.json', 'r') as f:
    data = json.load(f)

# Calculate the distribution of the lengths of the prompts ("training_sample")
lengths = []
for row in data:
    lengths.append(len(row['training_sample']))
plt.hist(lengths, bins=100)
plt.xlabel("Length of prompt")
plt.ylabel("Number of prompts")
plt.title("Distribution of the lengths of the prompts")
plt.show()

# Calculate the distribution of the lengths of the prompts in tokens using the tokenizer
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
lengths = []
for row in data:
    lengths.append(len(encoding.encode(row['training_sample'])))
plt.hist(lengths, bins=100)
plt.xlabel("Length of prompt in tokens")
plt.ylabel("Number of prompts")
plt.title("Distribution of the lengths of the prompts in tokens")
plt.show()

In [ ]:
from datasets import load_dataset

dataset_name = "magnus42/GPTWebScrapingPythonCode"
dataset = load_dataset(dataset_name, split="validation")
dataset